In [1]:
# Import library for scientific computing
import numpy as np
from scipy import stats
# Import library for data wrangling
import pandas as pd
# Import libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Import machine learning algorithm to utilize
from lightgbm import LGBMClassifier

# Importing modules for spliting data and cross validation
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, StratifiedKFold
# Importing performance metrics
from sklearn.metrics import roc_auc_score

# Set Max row and columns to have an overview of all features
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Reading the loan data
loan_df = pd.read_csv('../data/lending_club_loan_data.csv')

In [3]:
loan_df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,Not Verified,Jan-2015,Fully Paid,vacation,Vacation,26.24,Jun-1990,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,Not Verified,Jan-2015,Fully Paid,debt_consolidation,Debt consolidation,22.05,Jul-2004,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,Source Verified,Jan-2015,Fully Paid,credit_card,Credit card refinancing,12.79,Aug-2007,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,Not Verified,Nov-2014,Fully Paid,credit_card,Credit card refinancing,2.60,Sep-2006,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,Verified,Apr-2013,Charged Off,credit_card,Credit Card Refinance,33.95,Mar-1999,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"


### Data features meaning
- loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
- term: The number of payments on the loan. Values are in months and can be either 36 or 60.
- int_rate:	Interest Rate on the loan
- installment: The monthly payment owed by the borrower if the loan originates.
- grade: LC assigned loan grade
- sub_grade: LC assigned loan subgrade
- emp_title: The job title supplied by the Borrower when applying for the loan.*
- emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
- home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
- annual_inc: The self-reported annual income provided by the borrower during registration.
- verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
- issue_d: The month which the loan was funded
- loan_status: Current status of the loan
- purpose: A category provided by the borrower for the loan request.
- title: The loan title provided by the borrower
- zip_code: The first 3 numbers of the zip code provided by the borrower in the loan application.
- addr_state: The state provided by the borrower in the loan application
- dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
- earliest_cr_line: The month the borrower's earliest reported credit line was opened
- open_acc: The number of open credit lines in the borrower's credit file.
- pub_rec: Number of derogatory public records
- revol_bal: Total credit revolving balance
- revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
- total_acc: The total number of credit lines currently in the borrower's credit file
- initial_list_status: The initial listing status of the loan. Possible values are – W, F
- application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
- mort_acc:	Number of mortgage accounts.
- pub_rec_bankruptcies: Number of public record bankruptcies

In [4]:
def quick_preprocessing(df):
    # Feature Engineering
    ## Extract zip code from address feature
    df['zip_code'] = df.address.apply(lambda x: x[-5:])
    # ## convert date feature to date time feature
    df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
    ## Extract year from the earliest_cr_line feature
    df['earliest_cr_line'] = df.earliest_cr_line.dt.year.astype(int)
    
    # Dropping features
    ## charge off rates are extremely similar across all employment lengths. So we are going to drop the emp_length column.
    ## grade is just a sub feature of sub_grade, So we are goinig to drop it.
    ## issue_d will yield data leakage as it will show whether or not a loan would be issued when using our model.
    ## gotten zip code feature for address and contains only text component, so we are going to drop it.
    df.drop(columns=['emp_title', 'emp_length', 'title', 'grade', 'address', 'issue_d'], inplace=True)
    
    # Filling missing values with median value
    df['mort_acc'] = df['mort_acc'].fillna(df['mort_acc'].median().round())
    df['revol_util'] = df['revol_util'].fillna(df['revol_util'].median().round())
    df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(df['pub_rec_bankruptcies'].median().round())
    
    ## Mapping features
    term_values = {' 36 months': 36, ' 60 months': 60}
    df['term'] = df.term.map(term_values)
    ## Mapping target feature
    df['loan_status'] = df.loan_status.map({'Fully Paid':1, 'Charged Off':0})
    
    # #Change cattegorical features to categorical data type
    cat_columns =  ['sub_grade', 'verification_status', 'purpose', 'initial_list_status','application_type', 'home_ownership', 'zip_code']
    #Change some features to categorical data type
    for item in cat_columns:
        df[item] = df[item].astype("category").cat.codes +1

    # # Categorical encoding
    # dummies =
    # df = pd.get_dummies(df, columns=dummies, drop_first=True)
    return df, cat_columns

In [5]:
loan_df, cat_columns = quick_preprocessing(loan_df)

In [6]:
# Seperating matrix of features and target variables respectively
y =  loan_df['loan_status']
X = loan_df.drop(['loan_status'], axis=1)

# Splitting the data with a train:test ratio of 70:20
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

In [7]:
auc_scores = []
# Tackling overfitting with cross validation
## Stratified K-Fold Cross-Validation works perfectly well for Imbalanced Data
fold = StratifiedKFold(3)
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Initializing model
    lgb_model = LGBMClassifier()
    lgb_model.fit(X_train,y_train, categorical_feature=cat_columns)
    # Making predictions with the model
    predictions = lgb_model.predict(X_test)
    auc_scores.append(roc_auc_score(y_test, predictions))
    average_auc_scores = np.mean(auc_scores)
# print("AUC Score list: ", auc_scores)
print("Average AUC over splits: ", average_auc_scores)
    

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


Average AUC over splits:  0.7312331044587514


In [8]:
from deepchecks import Dataset
from deepchecks.suites import full_suite
ds_train = X_train.merge(y_train, left_index=True, right_index=True)
ds_test = X_test.merge(y_test, left_index=True, right_index=True)
ds_train = Dataset(ds_train, label="loan_status", cat_features=cat_columns)
ds_test =  Dataset(ds_test,  label="loan_status", cat_features=cat_columns)
suite = full_suite()


In [10]:
suite.run(train_dataset=ds_train, test_dataset=ds_test, model=lgb_model)

Full Suite:   0%|          | 0/36 [00:00<?, ? Check/s]